# RSI model
https://chatgpt.com/c/67f9fbc1-777c-800a-a56a-aadd83458b40

In [ ]:
# 📌 Cell 1: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# RSI Calculation
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [ ]:
# 📌 Cell 2: Load full training data
df = pd.read_csv("XAGUSD-H1-rates.csv", sep=r'\s+', engine='python')
df.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']

# Convert to datetime
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df = df.sort_values('Datetime')
df.reset_index(drop=True, inplace=True)

# Compute RSI
df['RSI'] = compute_rsi(df['Close'], period=14)

# Drop NaN rows from RSI calculation
df.dropna(inplace=True)


In [ ]:
# 📌 Cell 3: Labeling reversal (RSI 30/70)
def label_reversal_targets(data, future_window=5):
    labels = []
    for i in range(len(data) - future_window):
        rsi_now = data.iloc[i]['RSI']
        rsi_future = data.iloc[i+1:i+future_window+1]['RSI']

        if rsi_now < 30 and any(r > 35 for r in rsi_future):
            labels.append(1)  # Bullish reversal
        elif rsi_now > 70 and any(r < 65 for r in rsi_future):
            labels.append(1)  # Bearish reversal
        else:
            labels.append(0)
    return np.array(labels)

labels = label_reversal_targets(df)


In [ ]:
# 📌 Cell 4: Prepare input windows
SEQ_LEN = 30
features = ['Open', 'High', 'Low', 'Close', 'RSI']

X = []
for i in range(len(df) - SEQ_LEN - 5):  # 5 for future reversal window
    window = df.iloc[i:i+SEQ_LEN][features].values
    X.append(window)

X = np.array(X)
y = labels[:len(X)]

print("X shape:", X.shape)
print("y shape:", y.shape)


In [ ]:
# 📌 Cell 5: Train/Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

# LSTM model
model = Sequential([
    LSTM(64, input_shape=(SEQ_LEN, len(features))),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Training
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


In [ ]:
# 📌 Cell 6: Predict on rows1-30.csv
df_input = pd.read_csv("new-data-for-test/rows-30-from-20240503.csv", sep=r'\s+', engine='python')
df_input.columns = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread']
df_input['RSI'] = compute_rsi(df_input['Close'], period=14)
df_input.dropna(inplace=True)  # Drop NaNs from RSI

# Build input sample
sample = df_input[['Open', 'High', 'Low', 'Close', 'RSI']].values[-30:]
sample = np.expand_dims(sample, axis=0)  # Shape: (1, 30, 5)

# Predict
prediction = model.predict(sample)[0][0]
print("Reversal probability in next 5 candles:", round(prediction * 100, 2), "%")

if prediction > 0.5:
    print("⚠️ Likely reversal ahead!")
else:
    print("✅ No reversal detected.")
